<h1>Capstone Project - The Battle of Neighborhoods</h1>

<h3>Introduction</h3>

<p>Selecting the best location to open a mexican food restaurant in Manhattan, New York</p>


<p>The City of New York is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigrant history.</p>

<p>Mexican food is very popular around the world, its delicious flavour and spicy taste so starting a mexican food restaurant can be a great business opportunity, but you need to distinguish yourself from others to enjoy long-term success.</p>

<h3>Bussiness Problem</h3>

<p>My client wants to open his business in Manhattan area, so I focus on that borough during my analysis. We define potential neighborhood based on the number of mexican food restaurant which are operating right in each neighborhood. Manhattan has full potential but also is a very challenging district to open a business because of high competition. New mexican food restaurant should be open in an area that inadequate neighborhood in this way the bar can attract more customers. Therefore, this analysis necessary to ensure that we have enough customers and that we are not so close to other restaurants.</p>

<p>It is important to recognize that Manhattan is one of the most popular cities in NY, so it is diverse, with people of many different cultures and many bars, restaurants etc. so it is important to take the best decision in order to not to take risks when oppenning the restaurant.</p>

<h3>Data</h3>

<p>To identify the characteristics of our competitors' venues in Manhattan, we would first need to find out the number of mexican food restaurants in Manhattan currently and their location. We then used Google Map API to find their geographic coordinates based on their postal code addresses.</p>

<p>We will use a Dataset Neighborhood that has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.</p>

This report will be of most value to the following class of people:

    Investors who are looking to open a mexican food restaurant.
    People who already have a mexican food restaurant and want to expand business
    

<h3>Methodology</h3>

<ol><lu><h4>Importing libraries</h4></lu>

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographi

<lu><h4>Download and Explore Dataset</h4></lu>

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


<strong>
Tranforming the data into a pandas dataframe</strong>

In [5]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


<strong>
Using geopy library to get the latitude and longitude values of New York City.
</strong>

In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


<strong>Creating a map of New York with neighborhoods superimposed on top</strong>

In [9]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [10]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

<h4><strong>Foursquare venues</strong></h4>

In [12]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [14]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'DLXOA52GLJFUIWIHZNHAM32NHQBDAHRF3EJ2DEU54QM4MMOR'
CLIENT_SECRET = 'CVLJUI05UG20V2Q2GOBMHAUBT43ZHXVR3IISSX4MIWWYUJOO'
VERSION = '20201020'

In [15]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_mexican = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1c1941735')
newyork_venues_mexican.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Taqueria Sinaloense,40.874574,-73.910687,Mexican Restaurant
1,Marble Hill,40.876551,-73.91066,Estrellita Poblana V,40.879687,-73.906257,Mexican Restaurant
2,Marble Hill,40.876551,-73.91066,Mi Lindo San Miguelito,40.880023,-73.906488,Mexican Restaurant
3,Marble Hill,40.876551,-73.91066,Picante Picante Mexican Restaurant,40.878252,-73.902936,Mexican Restaurant
4,Marble Hill,40.876551,-73.91066,Guacamole,40.874511,-73.910708,Mexican Restaurant


In [16]:
newyork_venues_mexican.shape

(1625, 7)

In [17]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [18]:
map_newyork_mexican = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_mexican, 'red', map_newyork_mexican)

map_newyork_mexican

In [21]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [23]:
manhattan_grouped = newyork_venues_mexican.groupby('Neighborhood').count()
manhattan_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,47,47,47,47,47,47
Carnegie Hill,43,43,43,43,43,43
Central Harlem,26,26,26,26,26,26
Chelsea,48,48,48,48,48,48
Chinatown,49,49,49,49,49,49
Civic Center,47,47,47,47,47,47
Clinton,49,49,49,49,49,49
East Harlem,48,48,48,48,48,48
East Village,49,49,49,49,49,49


<h4><strong>Analyzing Each Neighborhood</strong></h4>

In [25]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_mexican[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_mexican['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Fast Food Restaurant,Food Stand,Food Truck,Ice Cream Shop,Japanese Restaurant,Latin American Restaurant,Mexican Restaurant,Sandwich Place,Seafood Restaurant,Taco Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [26]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Fast Food Restaurant,Food Stand,Food Truck,Ice Cream Shop,Japanese Restaurant,Latin American Restaurant,Mexican Restaurant,Sandwich Place,Seafood Restaurant,Taco Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.000000,0.021277,0.000000,0.085106,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.638298,0.000000,0.000000,0.234043,0.000000,0.000000
1,Carnegie Hill,0.000000,0.000000,0.023256,0.023256,0.000000,0.023256,0.000000,0.023256,0.000000,0.023256,0.000000,0.000000,0.023256,0.720930,0.000000,0.000000,0.139535,0.000000,0.000000
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.923077,0.000000,0.000000,0.038462,0.038462,0.000000
3,Chelsea,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.020833,0.020833,0.000000,0.020833,0.020833,0.562500,0.020833,0.020833,0.250000,0.020833,0.000000
4,Chinatown,0.000000,0.000000,0.000000,0.061224,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.775510,0.000000,0.000000,0.142857,0.000000,0.000000
5,Civic Center,0.000000,0.021277,0.000000,0.063830,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.723404,0.000000,0.000000,0.191489,0.000000,0.000000
6,Clinton,0.000000,0.000000,0.000000,0.040816,0.000000,0.000000,0.000000,0.020408,0.000000,0.040816,0.000000,0.000000,0.020408,0.714286,0.000000,0.000000,0.163265,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.000000,0.020833,0.000000,0.020833,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.812500,0.000000,0.000000,0.104167,0.000000,0.000000
8,East Village,0.000000,0.000000,0.000000,0.040816,0.000000,0.000000,0.040816,0.000000,0.000000,0.020408,0.000000,0.000000,0.020408,0.714286,0.000000,0.000000,0.163265,0.000000,0.000000
9,Financial District,0.000000,0.020833,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.229167,0.000000,0.000000


In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Bar,Fast Food Restaurant,Burger Joint,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
1,Carnegie Hill,Mexican Restaurant,Taco Place,Food Truck,Fast Food Restaurant,Burger Joint,Latin American Restaurant,Burrito Place,Chinese Restaurant,Bar,Caribbean Restaurant
2,Central Harlem,Mexican Restaurant,Taco Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant
3,Chelsea,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Seafood Restaurant,Sandwich Place,Latin American Restaurant,Japanese Restaurant,Tex-Mex Restaurant,Food Stand
4,Chinatown,Mexican Restaurant,Taco Place,Burrito Place,Latin American Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant


<strong>Clustering Neighborhoods
</strong>

In [30]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 3, 2, 4, 1, 3, 3, 1, 3, 4], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Mexican Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
1,Manhattan,Chinatown,40.715618,-73.994279,1,Mexican Restaurant,Taco Place,Burrito Place,Latin American Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Mexican Restaurant,Taco Place,Latin American Restaurant,American Restaurant,Caribbean Restaurant,Fast Food Restaurant,Bar,Burger Joint,Burrito Place,Chinese Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,1,Mexican Restaurant,Taco Place,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Mexican Restaurant,Food Truck,Taco Place,Tex-Mex Restaurant,Fast Food Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Midtown,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Latin American Restaurant,Tex-Mex Restaurant,Fast Food Restaurant,Cocktail Bar,Bar,Burger Joint
16,Murray Hill,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Tex-Mex Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant
21,Tribeca,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Bar,Burger Joint,Vegetarian / Vegan Restaurant,American Restaurant,Japanese Restaurant,Ice Cream Shop
24,West Village,Mexican Restaurant,Taco Place,Food Truck,Burrito Place,Ice Cream Shop,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Latin American Restaurant,Sandwich Place,Seafood Restaurant
27,Gramercy,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Food Stand,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant
33,Midtown South,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Latin American Restaurant,Food Stand,Fast Food Restaurant,Cocktail Bar,Bar,Burger Joint


In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Mexican Restaurant,Taco Place,Burrito Place,Latin American Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant
3,Inwood,Mexican Restaurant,Taco Place,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
4,Hamilton Heights,Mexican Restaurant,Food Truck,Taco Place,Tex-Mex Restaurant,Fast Food Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant
7,East Harlem,Mexican Restaurant,Taco Place,Food Truck,Burrito Place,Chinese Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Cocktail Bar
12,Upper West Side,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
18,Greenwich Village,Mexican Restaurant,Taco Place,Burrito Place,Latin American Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Cocktail Bar,Food Truck,Ice Cream Shop,Japanese Restaurant
22,Little Italy,Mexican Restaurant,Taco Place,Burrito Place,Bar,Latin American Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,Cocktail Bar,Burger Joint,Caribbean Restaurant
23,Soho,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Ice Cream Shop,Bar,Vegetarian / Vegan Restaurant,Latin American Restaurant,Japanese Restaurant,Tex-Mex Restaurant
31,Noho,Mexican Restaurant,Taco Place,Cocktail Bar,Latin American Restaurant,Burrito Place,Ice Cream Shop,Vegetarian / Vegan Restaurant,Bar,Burger Joint,Caribbean Restaurant


In [37]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Mexican Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
5,Manhattanville,Mexican Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
6,Central Harlem,Mexican Restaurant,Taco Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant
25,Manhattan Valley,Mexican Restaurant,Food Truck,Taco Place,Bar,Fast Food Restaurant,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
26,Morningside Heights,Mexican Restaurant,Taco Place,Bar,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar


In [38]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Mexican Restaurant,Taco Place,Latin American Restaurant,American Restaurant,Caribbean Restaurant,Fast Food Restaurant,Bar,Burger Joint,Burrito Place,Chinese Restaurant
8,Upper East Side,Mexican Restaurant,Taco Place,Fast Food Restaurant,Burger Joint,Latin American Restaurant,Burrito Place,Vegetarian / Vegan Restaurant,Bar,Caribbean Restaurant,Chinese Restaurant
9,Yorkville,Mexican Restaurant,Taco Place,Food Truck,Fast Food Restaurant,Burger Joint,Latin American Restaurant,Bar,Burrito Place,Caribbean Restaurant,Chinese Restaurant
10,Lenox Hill,Mexican Restaurant,Taco Place,Latin American Restaurant,Burrito Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant
11,Roosevelt Island,Mexican Restaurant,Taco Place,Latin American Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant
13,Lincoln Square,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
14,Clinton,Mexican Restaurant,Taco Place,Food Truck,Burrito Place,Fast Food Restaurant,Latin American Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant
19,East Village,Mexican Restaurant,Taco Place,Burrito Place,Cocktail Bar,Food Truck,Latin American Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant
20,Lower East Side,Mexican Restaurant,Taco Place,Cocktail Bar,Latin American Restaurant,Burrito Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant
30,Carnegie Hill,Mexican Restaurant,Taco Place,Food Truck,Fast Food Restaurant,Burger Joint,Latin American Restaurant,Burrito Place,Chinese Restaurant,Bar,Caribbean Restaurant


In [39]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Chelsea,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Seafood Restaurant,Sandwich Place,Latin American Restaurant,Japanese Restaurant,Tex-Mex Restaurant,Food Stand
28,Battery Park City,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Bar,Fast Food Restaurant,Burger Joint,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
29,Financial District,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Bar,Fast Food Restaurant,Burger Joint,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
34,Sutton Place,Mexican Restaurant,Taco Place,Burrito Place,Latin American Restaurant,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Bar,Burger Joint,Caribbean Restaurant
35,Turtle Bay,Mexican Restaurant,Taco Place,Burrito Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant
36,Tudor City,Mexican Restaurant,Taco Place,Burrito Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant
37,Stuyvesant Town,Mexican Restaurant,Taco Place,Burrito Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
38,Flatiron,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Food Stand,Fast Food Restaurant,Bar,Burger Joint,Caribbean Restaurant,Chinese Restaurant
39,Hudson Yards,Mexican Restaurant,Taco Place,Food Truck,Burrito Place,Fast Food Restaurant,Bar,Latin American Restaurant,Burger Joint,Caribbean Restaurant,Chinese Restaurant


<h3><strong>Results and Conclussion</strong></h3>

<p>We can see in maps and clusters that "Washington Heights Cluster 3" and Cluster 2 could be a good option if you want to open a mexican food restaurant, but in general there are many mexican restaurants. It can be done more detailed analysis by adding other factors such as transportation, demographics of inhabitants.</p>

<p>This analysis is performed on limited data. This may be right or may be wrong. But if good amount of data is available there is scope to come up with better results.
</p>

<p>Although all of the goals of this project were met there is definitely room for further improvement and development as noted below. However, the goals of the project were met and, with some more work, could easily be developed into a full application that could support the opening a business idea in an unknown location.
</p>